<a href="https://colab.research.google.com/github/ubinix-warun/mad-bootcamp-2024/blob/main/colab/lab_lc_l7a_tools_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LC-L7a-Lab: Tools/Agent

This code lab will guide you through the basics of LangChain Agent and Tools calling.

In [ ]:
!pip install -q langchain==0.2.2
!pip install -q langchain_community==0.2.3
!pip install -q langchain-openai==0.1.8
!pip install -q langgraph==0.0.64

!pip install -q requests==2.32.3
!pip install -q beautifulsoup4==4.12.2

## Prerequisite: Generate OpenAI key

Generate key from: https://platform.openai.com/api-keys

In [ ]:
import os

# TODO-1: put your generated openai key here
os.environ["OPENAI_API_KEY"] = ""

## Task1: Build KBank Job Search Tools

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

from bs4 import BeautifulSoup

In [ ]:
# model
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# prompt template
prompt_template = ChatPromptTemplate.from_messages(
    [("system",
"""
You are HTML content extractor.
A HTML text will be given, your task is to parse information into JSON with these following keys:
- job_title: string, a job title name.
- job_link: string, a url link to job description page, typically after "href="
- job_location: string, a job location.
- job_facility: string, a job facility.
answer as JSON object.
"""),
     ("user", "{html}")]
)

# parser
parser = JsonOutputParser()

html_job_extractor_chain = prompt_template | model | parser

In [ ]:
import requests, json
from langchain.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_core.tools import tool


class JobEntry(BaseModel):
    job_title: str = Field(description="job title name.")
    job_link: str = Field(description="url link to job description page.")
    job_location: str = Field(description="job location.")
    job_facility: str = Field(description="job facility.")
    job_description: str = Field(description="job description.")

# note: this run in sync (future works: change to async)

def get_job_description(url):
    response = requests.get("https://www.kasikorncareers.com"+url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    job_desc_content = soup.find('span', class_='jobdescription')
    return job_desc_content.get_text()

@tool
def search_job(query: str) -> List[JobEntry]:
    """searching the opening job."""

    # make GET request
    query_text = "+".join([e for e in query.split(" ") if e])
    response = requests.get("https://www.kasikorncareers.com/search/?q="+query_text)

    # using bs to extract only job content
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')
    html_job_contents = soup.find_all('td', class_='colTitle')

    # prase job content from HTML to json using LLM!
    jobs, max_job = [], 3
    for each_job_content in html_job_contents[:max_job]:
        try:
            job_detail = html_job_extractor_chain.invoke({"html": each_job_content})
            job_detail["job_description"] = get_job_description(job_detail["job_link"])
            jobs.append(job_detail)
        except:
            print("warning: error parsing html job")
            pass

    return jobs

In [ ]:
search_job("Software Developer")

In [ ]:
search_job("Analyst")

## Task2: Write Agent

In [ ]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

agent_executor = create_react_agent(model, [search_job], checkpointer=memory)
config = {"configurable": {"thread_id": "room-0001"}}

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi! I'm KhunThong")]}, config)

response["messages"]

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="what is my name")]}, config)

response["messages"][-1].content

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content= "Hello, Do you have Analyst Job opening?")]}, config)

response["messages"]

In [ ]:
response["messages"][-1].content